# 1. Wireless Networks Analysis

In this notebook I present results on performance evaluation of multihop wireless networks in terms of end-to-end delays, packets loss rates, busy rates and average buffer sizes.

Multihop wireless networks may be used for connecting remote LANs, providing internet access to villages or factories, they also may be found in many other use cases. However, in current research work I focus on networks, being used for data collecting from CCTV cameras or road RFID readers. Narrowing the use cases allows to extract distinctive properties of a wireless networks:

1. Data collection network may be assumed static, with a well-known topology and established routing tables.
2. Traffic is predominantely upstream, going from tree leaf nodes to the root.
3. The network is built from either radio relay, or Wi-Fi channels.

I consider the following possible types of wireless channels:

- radio relay channels, operating via narrow beams with low interference with Ethernet-like (P2P) protocol;
- basic long-range IEEE 802.11g channels with DCF access scheme;
- QoS-enabled IEEE 802.11e channels with EDCA aaccess.

Of course, these channel types are rather outdated. In 2020 many new standards exist, including IEEE 802.11ac/ax/ad. However, the basis of channel access is the same. I believe, that the methodology presented here is generic enough to take into account these more advanced versions.

> Main goal of this research is to find suitable queueing network model, that will properly describe the behaviour of the wireless network in terms of delays, packets loss and buffers sizes.

I need traffic models to works with. I consider two traffic sources: CCTV cameras and RFID readers. In both cases, UDP protocol is considered to be used as a transport (so no re-transmissions take place at transport layer). The traffic is described with `pcap` files:

- `data/traffic/video_XY.pcapng` - UDP/RTP traffic
- `data/traffic/rfid_XY.pcapng` - RFID traffic

To achieve the goal, I run through four steps:

1. Using inter-packet intervals from `.pcapng` files, I build exponential distribution and MAP for fitting them (let us denote this Poisson process or MAP as $A$). Several different MAPs with different number of states are found. Results of this step are written into `data/traffic/<TRAFFIC_NAME>_arrivals.json`, where `TRAFFIC_NAME` is the same name as for `pcapng` file.

2. Packet transmission time inside the wireless channel is not determined solely by the packet size. I use two ways to capture service time distributions:
    
    2.A. To take into account random delays and headers sizes, I run channel simulation with 0, 1 and 2 neighbour (interfering) stations. Each station is sending traffic from a `.pcapng` file. From this simulation, each packet service time is recorded into `data/channel/<TRAFFIC_NAME>_<CHAN>_service.csv` with three columns: `ID`, `SRC`, `DELAY`.  Then, for each of collection, I find fitting distributions: exponential (for all channels), PH (also for all channels) and MSP (for EDCA). Results are written into `data/channel/<TRAFFIC_NAME>_<CHAN>_service.json`.

    2.B. As an alternative to simulation, a channel can be modeled with a semi-Markov chain (however, in this case traffic is assumed to be saturated). Estimated distributions (also exponential, PH and MSP) are written into `data/channels/<TRAFFIC_NAME>_<CHAN>_service_sat.json`.

3. I run series of numerical experiments for all found traffic and service time models, varying the network length. From these experiments, I collect end-to-end delays, busy ratios, packets losses and queue sizes. Results are written into `data/networks/<TRAFFIC_NAME>_<CHAN>_<MODEL_NAME>.csv`, where `MODEL_NAME` can be `mm` (exponential and exponential), `mapXphY`, `mapXmspY` (`X` and `Y` denote the number of states). For saturated service model, the `MODEL_NAME` can take `mm_sat`, `mapXphY_sat` and so on.

4. I run wireless network simulation with the same `pcapng` files used for traffic. Results from the model are compared to the queueing results.

